In [15]:
import guidance
from guidance import models, gen, select, one_or_more, zero_or_more

## Setup

In [6]:
path = "llama-2-7b-chat.Q2_K.gguf"
llama2 = models.LlamaCpp(path)

Cannot use verbose=True in this context (probably CoLab). See https://github.com/abetlen/llama-cpp-python/issues/729
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count 

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

## Basics

In [7]:
llama2 + f"Do you want a joke or a poem? " + gen(stop=".")

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [13]:
llama2 + f"Do you want a joke or a poem? A " + select(["poem", "joke"])

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

## Context-Free Grammar

In [16]:
@guidance(stateless=True)
def number(lm):
    n = one_or_more(select(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]))
    # Allow for negative or positive numbers
    return lm + select(["-" + n, n])


@guidance(stateless=True)
def operator(lm):
    return lm + select(["+", "*", "**", "/", "-"])


@guidance(stateless=True)
def expression(lm):
    # Either
    # 1. A number (terminal)
    # 2. two expressions with an operator and optional whitespace
    # 3. An expression with parentheses around it
    return lm + select(
        [
            number(),
            expression()
            + zero_or_more(" ")
            + operator()
            + zero_or_more(" ")
            + expression(),
            "(" + expression() + ")",
        ]
    )

In [17]:
# without constraints
lm = llama2 + "Problem: Luke has a hundred and six balls. He then loses thirty six.\n"
lm += "Equivalent arithmetic expression: " + gen(stop="\n") + "\n"

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [18]:
# with constraints
grammar = expression()
lm = llama2 + "Problem: Luke has a hundred and six balls. He then loses thirty six.\n"
lm += "Equivalent arithmetic expression: " + grammar + "\n"

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [19]:
json_grammar = """
<json> ::= <object> | <array>
<object> ::= '{' <members> '}' | '{' '}'
<members> ::= <pair> | <pair> ',' <members>
<pair> ::= <string> ':' <value>
<array> ::= '[' <elements> ']' | '[' ']'
<elements> ::= <value> | <value> ',' <elements>
<value> ::= <string> | <number> | <object> | <array> | 'true' | 'false' | 'null'
<string> ::= '"' <chars> '"'
<number> ::= <int> | <int> '.' <int>
<chars> ::= /* any unicode characters except " or \, with escapes */
<int> ::= /* digits */
"""